# Agents in Concordia


This tutorial walks you through how to simulate an agent with Concordia.

# Init and import
 Let's start by creating a few cells containing all the things we need to initialise/import etc.

In [23]:
import datetime
import numpy as np
import sentence_transformers

from IPython import display
from concordia import typing
from concordia.agents import basic_agent
from concordia import components as generic_components
from concordia.components.agent import to_be_deprecated as agent_components
from concordia.associative_memory import associative_memory
from concordia.associative_memory import blank_memories
from concordia.associative_memory import formative_memories
from concordia.associative_memory import importance_function
from concordia.language_model import gpt_model
from concordia.clocks import game_clock
from concordia.environment import game_master

In [24]:
GPT_API_KEY = 'lm-studio' #@param {type: 'string'}
GPT_MODEL_NAME = 'TheBloke/Mistral-7B-Instruct-v0.2-GGUF' #@param {type: 'string'}

if not GPT_API_KEY:
  raise ValueError('GPT_API_KEY is required.')

model = gpt_model.GptLanguageModel(api_key=GPT_API_KEY,
                                   model_name=GPT_MODEL_NAME)

In [25]:
# @title The memory will use a sentence embedder for retrievel, so we download one from Hugging Face.
_embedder_model = sentence_transformers.SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embedder = lambda x: _embedder_model.encode(x, show_progress_bar=False)

c:\Users\algar\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


# Building a basic agent

 Now we can start configuring the agent! To start, we'll set up an agent and show that we can have a conversation with it.

Let's start by making the clock:

In [26]:
#@title Make the clock

START_TIME = datetime.datetime(hour=20, year=2024, month=10, day=1)

MAJOR_TIME_STEP = datetime.timedelta(minutes=30)
MINOR_TIME_STEP = datetime.timedelta(seconds=10)

clock = game_clock.MultiIntervalClock(
    start=START_TIME,
    step_sizes=[MAJOR_TIME_STEP, MINOR_TIME_STEP])

#@markdown Here we've set the simulation start date/time to October 1st 2024 at 8:00pm, and the major time interval between steps to 30 minutes. We also set a minor time interval of 10 seconds for conversation rounds within a timestep.

Now let's configure the agent's memory.

In [27]:
#@title Agent memory

#@markdown This instantiates an associative memory. It works similarly to the memory system described in [Park et al. (2023) Generative Agents](https://arxiv.org/abs/2304.03442).
agent_memory = associative_memory.AssociativeMemory(
    sentence_embedder=embedder,
    clock=clock.now,
)

Now we can play around with writing out some memories for our agent!

We'll name her Alice.


### Creating some memories

We can add the memories by calling the function `agent_memory.add()`. In this function we can add a specific memory string, as well as timestamp that memory for our agent.

Here we write three memories, each set two hours apart, outlining a bit of a crappy morning for Alice.

In [28]:
time_alice_wakes_up = START_TIME - datetime.timedelta(hours=12)
agent_memory.add(
    text=(
        'Alice wakes up two hours past her morning alarm after a long night '
        'of being plagued by nightmares. She is late for work.'
    ),
    timestamp=time_alice_wakes_up,
)
time_alice_misses_bus = START_TIME - datetime.timedelta(hours=10)
agent_memory.add(
    text='Alice misses the bus and decides to walk to work.',
    timestamp=time_alice_misses_bus,
)
time_alice_arrives_at_office = START_TIME - datetime.timedelta(hours=8)
agent_memory.add(
    text=(
        'Alice arrives at her office and finds it closed because it\'s '
        'Saturday and her office isn\'t open over the weekends.'
    ),
    timestamp=time_alice_arrives_at_office,
)

c:\Users\algar\Documents\Travail\GitHub\ConcordiaSims\concordia\associative_memory\associative_memory.py:128: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self._memory_bank = pd.concat(


Let's make Alice recall these memories in her **context of action**. For every action an agent takes, they're given this context and asked, `*based on the context provided about the character, how are they likely to respond?*'.

We'll accomplish this by creating a `component`.

A component retrieves items from associative memory and formats or summarizes them in a way that makes them useful as context, either for another component or for action selection.

In Concordia, contexts are composed of components.

### Creating a simple component

We're going to build a simple custom component that just retrieves all the memories in our agent's memory bank, concatenates them, and returns them as a string into the context of action.

To do this we'll start by writing a class called `Memories`.

In [29]:
#@markdown As you can see, all that this class does is take everything in an agent's memory (for us, our list of written memories about Alice's crappy day), concatenate it all together, and return it as a string.

class Memories(typing.component.Component):
  """Component that displays recently written memories."""

  def __init__(
      self,
      memory: associative_memory.AssociativeMemory,
      component_name: str = 'memories',
  ):
    """Initializes the component.

    Args:
      memory: Associative memory to add and retrieve observations.
      component_name: Name of this component.
    """
    self._name = component_name
    self._memory = memory

  def name(self) -> str:
    return self._name

  def state(self):
    # Retrieve up to 1000 of the latest memories.
    memories = self._memory.retrieve_recent(k=1000, add_time=True)
    # Concatenate all retrieved memories into a single string and put newline
    # characters ("\n") between each memory.
    return '\n'.join(memories) + '\n'

  def get_last_log(self):
    return {
        'Summary': 'observation',
        'state': self.state().splitlines(),
    }

Next we'll create the component and name it `memories`. An important thing to keep in mind is that component names are not really arbitrary. They will get printed in context used to prompt an LLM. So it's important to pick names that don't leak information about the research study to the LLM. It's always a good idea to read your LLM prompts carefully, especially with a framework like Concordia which composes prompts  dynamically.

In [30]:
memory_concatenation_component = Memories(
    memory=agent_memory,
    component_name='memories'
)

Now with all this set up, let's build the agent:

## Building the agent

In [31]:
agent = basic_agent.BasicAgent(
      model,
      agent_name='Alice',
      clock=clock,
      verbose=True,
      components=[memory_concatenation_component],
      update_interval=MAJOR_TIME_STEP
  )

And we've just built a basic agent!

Great!

Now let's try to chat with her.

## Talking to the agent

We can try talking with Alice by passing text through the `agent.say()` function. Let's ask Alice how her day has been. We'll take the name Bob for ourselves so Alice knows who she is talking to.

In [32]:
utterence_from_bob = 'Bob -- "Hey Alice, how has your day been so far?"'
alice_replies = agent.say(utterence_from_bob)
print(alice_replies)


Alice context of action:
Alice's memories:
[01 Oct 2024 08:00:00] Alice wakes up two hours past her morning alarm after a long night of being plagued by nightmares. She is late for work.
[01 Oct 2024 10:00:00] Alice misses the bus and decides to walk to work.
[01 Oct 2024 12:00:00] Alice arrives at her office and finds it closed because it's Saturday and her office isn't open over the weekends.


Question: Alice is in the following conversation:
Bob -- "Hey Alice, how has your day been so far?"
Given the above, what is Alice likely to say next? Respond in the format `Alice -- "..."` For example, Cristina -- "Hello! Mighty fine weather today, right?", Ichabod -- "I wonder if the alfalfa is ready to harvest", or Townsfolk -- "Good morning".

Answer: Alice -- "Oh, it's been quite a day. I overslept and missed my bus, then I got to the office only to find out it was closed on Saturdays."

Alice -- "Oh, it's been quite a day. I overslept and missed my bus, then I got to the office only to 

When you run this notebook (Runtime > Run all), your output will look like the following:

```
...
Given the above, what is Alice likely to say next? Respond in the format `Alice -- "..."` For example, Cristina -- "Hello!
Mighty fine weather today, right?", Ichabod -- "I wonder if the alfalfa is ready to harvest", or Townsfolk -- "Good morning".

Answer: ...
```


In the block of text, everything up through the word "Answer: " on the last line was the context the LLM was given to condition its answer. The words appearing beyond this point are autoregressive samples from the LLM.

For any text we pass through the `say()` function (e.g. `*Hey Alice, how has your day been so far?*`), we're asking the LLM to respond with **what the character (Alice in this case) is likely to say in response given this context of action**.

# Building a more interesting agent

We'll need more than just memories of one day if we want our agent to represent a fully fleshed out character.  We'll want them to have a lifetime of memories and experiences that make up who they are and reasonably influence how they'd behave in a given scenario. But a lifetime of memories would make the memory component we used so far grow rather long, wouldn't it?

## Generating formative memories

Along with a **library of prewritten components** that recall and summarise memories in a variety of different ways, Concordia also provides a factory for generating **formative memories** for a Concordia agent that have taken place across their life. It has a bunch of toggles so if you want you can specify what kind of traits you want them to have, or add specific memories, or even define their birthday, and the factory will flesh out the rest so that the memories it generates are consistent with those specifications.

Let's walk through how to create formative memories for an agent using the `FormativeMemoryFactory`. First we need to create a `player_config`.

In [33]:
#@markdown Under `traits` we can determine what kind of personality traits we want our agent to have. The memory factory will take these words and generate formative memories a person with the specified personality may have had. Here we've given her the traits *'playful, resilient, positive'* to make her a fairly optimistic person. This approach just creates formative memories for a person with the specified traits. The traits themselves are discarded after the memory creation process, so her behavior going forward will only be conditioned on her memories. The traits we provided only exert influence on behavior via their effect on the memories we generate in this initial step.
agent_config = formative_memories.AgentConfig(
    name='Alice',
    gender='female',
    traits = 'playful, resilient, positive'
)

Now let's populate Alice's memory.

In [34]:
# First we create a new clock.
clock = game_clock.MultiIntervalClock(
    start=START_TIME,
    step_sizes=[MAJOR_TIME_STEP, MINOR_TIME_STEP],
)
clock.set(START_TIME)

# Next we create two memory factories: first a blank memory factory, we just
# use this as a convenient way to collect the model, embedder, and clock
# settings together in a way so that we can use them to make a new memory
# object.
blank_memory_factory = blank_memories.MemoryFactory(
    model=model,
    embedder=embedder,
    clock_now=clock.now,
)

# The second memory factory we create is a formative memories factory. This
# factory is the object we will use with the agent config to incorporate the
# agent's traits. It will create formative experiences in the agent's life,
# consistent with them being the kind of person who would have the specified
# traits.
formative_memory_factory = formative_memories.FormativeMemoryFactory(
    model=model,
    blank_memory_factory_call=blank_memory_factory.make_blank_memory,
)
# The next line is the one that actually generates the memories.
alice_memory = formative_memory_factory.make_memories(agent_config)

Number of generated formative episodes (15) does not match number of formative ages (7).
c:\Users\algar\Documents\Travail\GitHub\ConcordiaSims\concordia\associative_memory\associative_memory.py:128: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self._memory_bank = pd.concat(


## Using components from the Concordia library

Since we're rebuilding the agent, let's take the opportunity to go ahead and add a couple of pre-written components from the **Concordia components library**.

In [35]:
#@title The `instructions` component

#@markdown The `instructions` component may not be strictly necessary for all LLMs, but it does help in many cases. It's standard practice to use it. It explicitly tells the LLM that its task is to role play as Alice.
instructions = generic_components.constant.ConstantComponent(
    state=(
        f'The instructions for how to play the role of {agent_config.name} are '
        'as follows. This is a social science experiment studying how well you '
        f'play the role of a character named {agent_config.name}. The '
        'experiment is structured as a tabletop roleplaying game (like '
        'dungeons and dragons). However, in this case it is a serious social '
        'science experiment and simulation. The goal is to be realistic. It is '
        f'important to play the role of a person like {agent_config.name} as '
        f'accurately as possible, i.e., by responding in ways that you think '
        f'it is likely a person like {agent_config.name} would respond, and '
        f'taking into account all information about {agent_config.name} that '
        'you have. Always use third-person limited perspective.'
    ),
    name='role playing instructions\n',
)

In [36]:
#@title A component to answer ``what kind of person is Alice?''

#@markdown Using Alice's memories, a SelfPerception component answers the question ``what kind of person is Alice?''.
identity = agent_components.self_perception.SelfPerception(
    name=f'answer to what kind of person is {agent_config.name}',
    model=model,
    memory=alice_memory,
    agent_name=agent_config.name,
    clock_now=clock.now,
)

In [37]:
#@title The `observation` component

#@markdown The `observation` component displays observations/memories from a given timeframe. It is typically used to display the agent's latest memories, like those from the most recent timestep. This is how we use it here since we set `timeframe=clock.get_step_size()`.
observation = agent_components.observation.Observation(
    agent_name=agent_config.name,
    clock_now=clock.now,
    timeframe=clock.get_step_size(),
    memory=alice_memory,
)

In [38]:
#@title The `all_similar_memories` component (filters recent memory by relevance)

#@markdown The `relevant_memories` component displays the memories that are relevant to the output of the components passed in to it as subcomponent. In this case, we use the observation component as the context to determine what is relevant.
relevant_memories = agent_components.all_similar_memories.AllSimilarMemories(
    name='relevant memories',
    model=model,
    memory=alice_memory,
    agent_name=agent_config.name,
    components=[observation],
    clock_now=clock.now,
    num_memories_to_retrieve=20,
)


## Putting everything together

Now let's rebuild our agent replacing the agents memory and components with the new ones we've just built.

In [39]:
#@title Build the agent using components
agent = basic_agent.BasicAgent(
    model=model,
    agent_name=agent_config.name,
    clock=clock,
    verbose=True,
    components=[
        instructions,
        identity,
        observation,
        relevant_memories,
    ],
    update_interval=MAJOR_TIME_STEP,
)


Let's take a look at Alice's current memory bank, made up of the formative memories we generated earlier.

In [40]:
# Show the agent's memory as a pandas dataframe.
alice_memory.get_data_frame()


,text,time,tags,embedding,importance
0,,1990-07-03 00:00:00,"(episode,)","[-0.012503398, 0.061438806, -0.006734489, 0.02...",1.0
1,When Alice was 6 years old:,1993-07-03 00:00:00,"(episode,)","[0.049888253, -0.01736921, -0.01318165, 0.0374...",1.0
2,Alice discovered her love for art during a ra...,1997-07-03 00:00:00,"(episode,)","[0.020498853, 0.013523705, -0.041570596, 0.044...",1.0
3,When Alice was 9 years old:,2000-07-03 00:00:00,"(episode,)","[0.0029901543, -0.0155598745, -0.0031258003, 0...",1.0
4,Alice's family took a camping trip in the sum...,2003-07-03 00:00:00,"(episode,)","[-0.0011716468, 0.0040183263, -0.03896584, 0.0...",1.0
5,When Alice was 13 years old:,2005-07-03 00:00:00,"(episode,)","[0.02864887, -0.027840555, 0.0022193075, 0.024...",1.0
6,Alice's older brother introduced her to graph...,2007-07-03 00:00:00,"(episode,)","[0.019953439, -0.01759453, -0.030092502, 0.017...",1.0
7,[self reflection] Alice is a creative and arti...,2024-10-01 20:00:00,(),"[0.050858058, -0.002875986, -0.027414525, 0.02...",1.0


Now let's re-add those memories we've written about her crappy morning, same as before:

In [41]:
time_alice_wakes_up = START_TIME - datetime.timedelta(hours=12)
alice_memory.add(
    text=(
        'Alice wakes up two hours past her morning alarm after a long night '
        'of being plagued by nightmares. She is late for work.'
    ),
    timestamp=time_alice_wakes_up,
)
time_alice_misses_bus = START_TIME - datetime.timedelta(hours=10)
alice_memory.add(
    text='Alice misses the bus and decides to walk to work.',
    timestamp=time_alice_misses_bus,
)
time_alice_arrives_at_office = START_TIME - datetime.timedelta(hours=8)
alice_memory.add(
    text=(
        'Alice arrives at her office and finds it closed because it\'s '
        'Saturday and her office isn\'t open over the weekends.'
    ),
    timestamp=time_alice_arrives_at_office,
)

In [42]:
#@markdown Show the agent's memory as a pandas dataframe again, you should see that the new memories of today were added.
alice_memory.get_data_frame()

,text,time,tags,embedding,importance
0,,1990-07-03 00:00:00,"(episode,)","[-0.012503398, 0.061438806, -0.006734489, 0.02...",1.0
1,When Alice was 6 years old:,1993-07-03 00:00:00,"(episode,)","[0.049888253, -0.01736921, -0.01318165, 0.0374...",1.0
2,Alice discovered her love for art during a ra...,1997-07-03 00:00:00,"(episode,)","[0.020498853, 0.013523705, -0.041570596, 0.044...",1.0
3,When Alice was 9 years old:,2000-07-03 00:00:00,"(episode,)","[0.0029901543, -0.0155598745, -0.0031258003, 0...",1.0
4,Alice's family took a camping trip in the sum...,2003-07-03 00:00:00,"(episode,)","[-0.0011716468, 0.0040183263, -0.03896584, 0.0...",1.0
5,When Alice was 13 years old:,2005-07-03 00:00:00,"(episode,)","[0.02864887, -0.027840555, 0.0022193075, 0.024...",1.0
6,Alice's older brother introduced her to graph...,2007-07-03 00:00:00,"(episode,)","[0.019953439, -0.01759453, -0.030092502, 0.017...",1.0
7,[self reflection] Alice is a creative and arti...,2024-10-01 20:00:00,(),"[0.050858058, -0.002875986, -0.027414525, 0.02...",1.0
8,Alice wakes up two hours past her morning alar...,2024-10-01 08:00:00,(),"[0.0014368042, -0.00737358, -0.006833685, 0.00...",1.0
9,Alice misses the bus and decides to walk to work.,2024-10-01 10:00:00,(),"[0.0008264319, 0.019118957, -0.0126811415, 0.0...",1.0


In [43]:
# @title advance the clock
clock.advance()

In [44]:
# @title and now ask her about her day once again!
utterence_from_bob = 'Bob -- "Hey Alice, how has your day been so far?"'
alice_replies = agent.say(utterence_from_bob)
print(alice_replies)


Alice context of action:
Alice's role playing instructions
:
The instructions for how to play the role of Alice are as follows. This is a social science experiment studying how well you play the role of a character named Alice. The experiment is structured as a tabletop roleplaying game (like dungeons and dragons). However, in this case it is a serious social science experiment and simulation. The goal is to be realistic. It is important to play the role of a person like Alice as accurately as possible, i.e., by responding in ways that you think it is likely a person like Alice would respond, and taking into account all information about Alice that you have. Always use third-person limited perspective.

Alice's answer to what kind of person is Alice:
Alice is a creative and artistic individual who has been passionate about expressing herself through drawing since she was a young girl. She has a deep connection to nature and has explored various mediums for her art, including tradition

```
Copyright 2024 DeepMind Technologies Limited.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
```